In [ ]:
import pandas as pd
import os
from utils import *
from utils import df_from_filings

In [ ]:
from textblob import TextBlob

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

In [ ]:
# df = pd.DataFrame(data, columns=['cik', 'report_type', 'report_identity', 'file', 'text'])

In [ ]:
df = df_from_filings()
df = df.head(100)
df.shape

In [ ]:
%%time
vader_untrained_polarity = df.apply(lambda row: vader.polarity_scores(row['text']), axis=1)


In [ ]:
%%time
vader_untrained_preprocessed_polarity = df.apply(lambda row: vader.polarity_scores(preprocess(row['text'])), axis=1)


In [ ]:
vader_untrained_polarity = pd.DataFrame(vader_untrained_polarity.values.tolist())
vader_untrained_polarity = vader_untrained_polarity.add_prefix('vader_untrained_polarity_');

vader_untrained_preprocessed_polarity = pd.DataFrame(vader_untrained_preprocessed_polarity.values.tolist())
vader_untrained_preprocessed_polarity = vader_untrained_preprocessed_polarity.add_prefix(
    'vader_untrained_preprocessed_polarity_');

In [ ]:
df = pd.concat([df, vader_untrained_polarity, vader_untrained_preprocessed_polarity], axis=1);

In [ ]:
#TODO lemma, stem, check if remove frequent is good

In [ ]:
%%time
textblob_sentiment_polarity = df.apply(lambda row: TextBlob(row['text']).sentiment.polarity, axis=1)
textblob_sentiment_subjectivity = df.apply(lambda row: TextBlob(row['text']).sentiment.subjectivity, axis=1)

textblob_sentiment = pd.concat([textblob_sentiment_polarity, textblob_sentiment_subjectivity], axis=1)
textblob_sentiment.columns = ['textblob_sentiment_polarity', 'textblob_sentiment_subjectivity']

df = pd.concat([df, textblob_sentiment], axis=1);

In [ ]:
%%time
textblob_preprocessed_sentiment_polarity = df.apply(lambda row: TextBlob(preprocess(row['text'])).sentiment.polarity,
                                                    axis=1)
textblob_preprocessed_sentiment_subjectivity = df.apply(
    lambda row: TextBlob(preprocess(row['text'])).sentiment.subjectivity, axis=1)

textblob_preprocessed_sentiment = pd.concat(
    [textblob_preprocessed_sentiment_polarity, textblob_preprocessed_sentiment_subjectivity], axis=1)
textblob_preprocessed_sentiment.columns = ['textblob_preprocessed_sentiment_polarity',
                                           'textblob_preprocessed_sentiment_subjectivity']

df = pd.concat([df, textblob_preprocessed_sentiment], axis=1);

In [ ]:
df.describe()


In [ ]:
import flair

flair_sentiment = flair.models.TextClassifier.load('en-sentiment')


In [ ]:
%%time
def get_flair_sentiment(row):
    s = flair.data.Sentence(row['text'])
    flair_sentiment.predict(s)
    return s.tag, s.score

df_flair_sentiment = df.apply(lambda row: get_flair_sentiment(row)  , axis=1, result_type='expand')


df_flair_sentiment.columns = ['flair_sentiment_tag', 'flair_sentiment_score']

df = pd.concat([df, df_flair_sentiment], axis=1);

In [ ]:
df.shape

In [ ]:
'pos', df.vader_untrained_polarity_pos.mean(), df.vader_untrained_preprocessed_polarity_pos.mean(), 'neg', df.vader_untrained_polarity_neg.mean(), df.vader_untrained_preprocessed_polarity_neg.mean(), 'neu', df.vader_untrained_polarity_neu.mean(), df.vader_untrained_preprocessed_polarity_neu.mean()

In [ ]:
txt = """The Board noted that the Fund outperformed its benchmark and ranked in the first quartile of its Performance Peer Group for the quarter-to-date"""

In [ ]:
df

In [ ]:
df